# Projeto 2 - Ciência dos Dados

Nome: Luiz Felipe Domingues Valente

Nome: João Guilherme Cintra de Freitas Almeida

Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
# Classificador automático de sentimento


## Preparando o ambiente no jupyter:

In [37]:
%%capture

#Instalando o tweepy
!pip install tweepy
!pip install emoji

In [38]:
import tweepy
import math
import os.path
import pandas as pd
import json
from random import shuffle
import numpy as np
import re 
from emoji import UNICODE_EMOJI

___
## Autenticando no  Twitter

* Conta: ***[Preencha aqui o id da sua conta. Ex: @fulano ]***

In [39]:
#Dados de autenticação do twitter:

# Coloque aqui o identificador da conta no twitter: @jofenina

#leitura do arquivo no formato JSON
# with open('auth.pass') as fp:    
#     data = json.load(fp)

#Configurando a biblioteca. Não modificar
# auth = tweepy.OAuthHandler(data['consumer_key'], data['consumer_secret'])
# auth.set_access_token(data['access_token'], data['access_token_secret'])

___
## Etapas do projeto:

### Escolha de um produto e coleta das mensagens


In [40]:
#Produto escolhido:
produto = 'snickers'

#Quantidade mínima de mensagens capturadas:
n = 500
#Quantidade mínima de mensagens para a base de treinamento:
t = 300

#Filtro de língua, escolha uma na tabela ISO 639-1.
lang = 'pt'

Capturando os dados do twitter:

In [41]:
#Cria um objeto para a captura
# api = tweepy.API(auth)

#Inicia a captura, para mais detalhes: ver a documentação do tweepy
# i = 1
# msgs = []
# for msg in tweepy.Cursor(api.search, q=produto, lang=lang, tweet_mode="extended").items():    
#     msgs.append(msg.full_text.lower())
#     i += 1
#     if i > n:
#         break

#Embaralhando as mensagens para reduzir um possível viés
# shuffle(msgs)

Salvando os dados em uma planilha Excel:

In [42]:
#Verifica se o arquivo não existe para não substituir um conjunto pronto
# if not os.path.isfile('./{0}.xlsx'.format(produto)):
    
    #Abre o arquivo para escrita
#     writer = pd.ExcelWriter('{0}.xlsx'.format(produto))

    #divide o conjunto de mensagens em duas planilhas
#     dft = pd.DataFrame({'Treinamento' : pd.Series(msgs[:t])})
#     dft.to_excel(excel_writer = writer, sheet_name = 'Treinamento', index = False)

#     dfc = pd.DataFrame({'Teste' : pd.Series(msgs[t:])})
#     dfc.to_excel(excel_writer = writer, sheet_name = 'Teste', index = False)

    #fecha o arquivo
#     writer.save()

___
### Montando o Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

# Limpeza

In [43]:
tweets_tr_sujos = pd.read_excel("snickers.xlsx")
tweets_tr_sujos.head()

,Treinamento,Relevância
0,@tiagoatoc nunca pensei que me mandasses comer...,0
1,"credo q eu acordei sem paciência, e já to come...",0
2,samyra me deu batata frita com cheddar e bacon...,1
3,crlh acabei de descobrir q tem sorvete de snic...,1
4,@yenexiel ninguém compra snickers pra mim quan...,0


In [44]:
treinamento = tweets_tr_sujos.loc[:, "Treinamento"]
rel=tweets_tr_sujos.loc[:, "Relevância"]
treinamento.head()

0    @tiagoatoc nunca pensei que me mandasses comer...
1    credo q eu acordei sem paciência, e já to come...
2    samyra me deu batata frita com cheddar e bacon...
3    crlh acabei de descobrir q tem sorvete de snic...
4    @yenexiel ninguém compra snickers pra mim quan...
Name: Treinamento, dtype: object

In [45]:
def cleanup(text):
    
#  Limpeza de caracteres

    punctuation = '[!\-.:?;@]'
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, ' ', text)
    return text_subbed    

In [46]:
t_limpos1 = []
t_limpos2 = []
t_limpos = []
for linha in treinamento:
    b=linha.lower()
    t_limpos1.append(cleanup(b))
for e in t_limpos1:
    a = e.strip()
    t_limpos2.append(a)
for s in t_limpos2:
    z = ' '.join(s.split())
    t_limpos.append(z)
t_limpos[0:5]

['tiagoatoc nunca pensei que me mandasses comer snickers antes de dormir sinceramente',
 'credo q eu acordei sem paciência, e já to comendo um snickers de café da manhã',
 'samyra me deu batata frita com cheddar e bacon, e evelyn me deu snickers, amo minhas amigas',
 'crlh acabei de descobrir q tem sorvete de snickers eu preciso pra ontem',
 'yenexiel ninguém compra snickers pra mim quando tô triste']

In [47]:
# search your emoji
def is_emoji(s):
    return s in UNICODE_EMOJI

# add space near your emoji
def add_space(text):
    return ''.join(' ' + char if is_emoji(char) else char for char in text).strip()

results=[add_space(text) for text in t_limpos]

print(results)

['tiagoatoc nunca pensei que me mandasses comer snickers antes de dormir sinceramente', 'credo q eu acordei sem paciência, e já to comendo um snickers de café da manhã', 'samyra me deu batata frita com cheddar e bacon, e evelyn me deu snickers, amo minhas amigas', 'crlh acabei de descobrir q tem sorvete de snickers eu preciso pra ontem', 'yenexiel ninguém compra snickers pra mim quando tô triste', 'se tu roubar uma birosca dentro da favela os cara da firma de dá um sapeca e tu leva tiro nas mãos o dimenor chicoteado deu sorte mas ainda assim saiu caro esse snickers  😂 https //t co/pirvh7zgb8', 'rt thaii_xx sério, o anúncio da snickers no yt tá mt chato, eles fazem o bgl pra vender e na real todo mundo vai é ficar c raiva do choco…', 'rt amandinhavr eu esperando a propaganda insuportável de snickers acabar de passar no vídeo https //t co/d0zqzsmpgu', 'não aguento mais esses anúncios da snickers, nao da nem pra pular', 'a dor do preço do snickers ter aumentado', 'fiquei vendo propaganda 

# Tabela com limpeza e relevância

In [48]:
t_tr = pd.DataFrame({"Treinamento": results,"Relevância": rel})
t_tr.head()

,Treinamento,Relevância
0,tiagoatoc nunca pensei que me mandasses comer ...,0
1,"credo q eu acordei sem paciência, e já to come...",0
2,samyra me deu batata frita com cheddar e bacon...,1
3,crlh acabei de descobrir q tem sorvete de snic...,1
4,yenexiel ninguém compra snickers pra mim quand...,0


# Tweets Relevantes Limpos e frequência de suas Palavras

In [49]:
t_rel= t_tr[(t_tr.Relevância==1)]
t_rel.head()

,Treinamento,Relevância
2,samyra me deu batata frita com cheddar e bacon...,1
3,crlh acabei de descobrir q tem sorvete de snic...,1
6,"rt thaii_xx sério, o anúncio da snickers no yt...",1
7,rt amandinhavr eu esperando a propaganda insup...,1
8,"não aguento mais esses anúncios da snickers, n...",1


In [50]:
serie_tweets = ' '.join(t_rel.loc[:,"Treinamento"])
a = pd.Series(serie_tweets.split())
a.head()

0    samyra
1        me
2       deu
3    batata
4     frita
dtype: object

In [51]:
a.value_counts().head()

snickers    166
de           92
eu           69
do           62
e            55
dtype: int64

In [52]:
tabela_tweets_relativa = a.value_counts(True)
tabela_tweets_relativa.head()

snickers    0.065252
de          0.036164
eu          0.027123
do          0.024371
e           0.021619
dtype: float64

In [53]:
imprime = 4
for palavra in tabela_tweets_relativa.index:
    if imprime:
        print(tabela_tweets_relativa[palavra])
        imprime-=1

0.06525157232704402
0.036163522012578615
0.027122641509433963
0.02437106918238994


In [113]:
log=[]
for frase in t_rel.Treinamento:
    frase_d=frase.split()
    freq_log = 0 
    for palavra in frase_d:
        if palavra in tabela_tweets_relativa:
            freq_log += np.log10(tabela_tweets_relativa[palavra])
            log.append(freq_log)
log

[-3.4055171069763763,
 -5.44930637793516,
 -8.00972544489728,
 -11.114212556209674,
 -14.51972966318605,
 -16.749155511106746,
 -20.154672618083122,
 -21.819827035565254,
 -25.22534414254163,
 -26.89049856002376,
 -30.296015667000137,
 -32.339804937958924,
 -34.90022400492104,
 -37.226559865849794,
 -39.728986985834226,
 -42.83347409714662,
 -46.238991204122996,
 -3.4055171069763763,
 -6.20897422262479,
 -7.650703502255611,
 -11.056220609231987,
 -13.285646457152682,
 -15.788073577137114,
 -18.71646942939383,
 -20.15819870902465,
 -21.343607727960972,
 -22.910275744200092,
 -26.315792851176468,
 -28.15310823408585,
 -31.558625341062225,
 -2.104487111312395,
 -5.20897422262479,
 -8.137370074881504,
 -10.011408264815625,
 -12.302982019485164,
 -14.140297402394545,
 -15.325706421330866,
 -17.199744611264986,
 -19.70217173124942,
 -22.50562884689783,
 -25.21217594953819,
 -28.140571801794902,
 -31.068967654051615,
 -33.99736350630833,
 -35.87140169624245,
 -38.799797548499164,
 -40.6371129

# Tweets não Relevantes Limpos e frequência de suas Palavras

In [54]:
t_n_rel= t_tr[(t_tr.Relevância==0)]
t_n_rel.head()

,Treinamento,Relevância
0,tiagoatoc nunca pensei que me mandasses comer ...,0
1,"credo q eu acordei sem paciência, e já to come...",0
4,yenexiel ninguém compra snickers pra mim quand...,0
5,se tu roubar uma birosca dentro da favela os c...,0
15,a casa do jovem surtado se resume a pacotes de...,0


In [55]:
serie_tweets_n_rel = ' '.join(t_n_rel.loc[:,"Treinamento"])
a_n_rel = pd.Series(serie_tweets_n_rel.split())
a.head()

0    samyra
1        me
2       deu
3    batata
4     frita
dtype: object

In [56]:
a_n_rel.value_counts().head()

snickers    101
e            64
um           54
de           51
o            40
dtype: int64

In [57]:
tabela_tweets_relativa_n_rel = a_n_rel.value_counts(True)
tabela_tweets_relativa_n_rel.head()

snickers    0.051347
e           0.032537
um          0.027453
de          0.025928
o           0.020336
dtype: float64

In [58]:
imprime = 4
for palavra in tabela_tweets_relativa_n_rel.index:
    if imprime:
        print(tabela_tweets_relativa_n_rel[palavra])
        imprime-=1

0.05134722928317234
0.03253685815963396
0.027452974072191154
0.02592780884595831


In [59]:
dic_log = {}
for frase in t_tr.Treinamento:
    frase_d=frase.split()
    freq_log = 0 
    for palavra in frase_d:
        if palavra in tabela_tweets_relativa:
            freq_log += np.log10(tabela_tweets_relativa[palavra])
    dic_log[frase] = freq_log
dic_log

{'tiagoatoc nunca pensei que me mandasses comer snickers antes de dormir sinceramente': -11.66216960872919,
 'credo q eu acordei sem paciência, e já to comendo um snickers de café da manhã': -25.814984799834072,
 'samyra me deu batata frita com cheddar e bacon, e evelyn me deu snickers, amo minhas amigas': -46.238991204122996,
 'crlh acabei de descobrir q tem sorvete de snickers eu preciso pra ontem': -31.558625341062225,
 'yenexiel ninguém compra snickers pra mim quando tô triste': -18.545159958407677,
 'se tu roubar uma birosca dentro da favela os cara da firma de dá um sapeca e tu leva tiro nas mãos o dimenor chicoteado deu sorte mas ainda assim saiu caro esse snickers  😂 https //t co/pirvh7zgb8': -56.20956572551127,
 'rt thaii_xx sério, o anúncio da snickers no yt tá mt chato, eles fazem o bgl pra vender e na real todo mundo vai é ficar c raiva do choco…': -72.48295125472242,
 'rt amandinhavr eu esperando a propaganda insuportável de snickers acabar de passar no vídeo https //t co/

___
### Verificando a performance

Agora você deve testar o seu classificador com a base de Testes.

___
### Concluindo

## Aperfeiçoamento:

Os trabalhos vão evoluir em conceito dependendo da quantidade de itens avançados:

* Limpar: \n, :, ", ', (, ), etc SEM remover emojis
* Corrigir separação de espaços entre palavras e emojis ou emojis e emojis
* Propor outras limpezas e transformações que não afetem a qualidade da informação ou classificação
* Criar categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante (3 categorias: C, mais categorias conta para B)
* Explicar por que não posso usar o próprio classificador para gerar mais amostras de treinamento
* Propor diferentes cenários para Naïve Bayes fora do contexto do projeto
* Sugerir e explicar melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* Montar um dashboard que periodicamente realiza análise de sentimento e visualiza estes dados

# Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**